In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_image_dir = 'data/train'
validation_image_dir = 'data/validation'
test_image_dir = "data/test"

In [ ]:
IMAGE_HEIGHT, IMAGE_WIDTH = 256, 256
BATCH_SIZE = 128
TRAIN_IMGS = 20000
VAL_IMGS = 5000
MAX_TRAIN_STEPS = 200
STEP_COUNT = min(MAX_TRAIN_STEPS, TRAIN_IMGS//BATCH_SIZE)
NB_EPOCHS = 30


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from tensorflow.keras.applications.vgg16 import VGG16

if K.image_data_format() == 'channels_first':
    input_shape = (3, IMAGE_WIDTH, IMAGE_HEIGHT)
else:
    input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
    
model = Sequential()

model = VGG16(include_top=False, input_shape=(256, 256, 3))
for layer in model.layers:
    layer.trainable = False
    
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model = Model(inputs=model.inputs, outputs=output)



In [ ]:
import pandas as pd
pd.options.display.max_rows = 4000
tf.keras.utils.plot_model(model,'arch.png', show_shapes=True)

In [ ]:
SGD_OPTIMIZER = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)

model.compile(loss='binary_crossentropy',
            optimizer=SGD_OPTIMIZER,
            metrics=['accuracy'])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, verbose=1, mode='min', epsilon=0.0001, cooldown=2, min_lr=1e-6)
early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10)
callbacks_list = [early, reduce_lr]

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 15, 
    width_shift_range = 0.1, 
    height_shift_range = 0.1, 
    shear_range = 0.01,
    zoom_range = [0.9, 1.25],  
    horizontal_flip = True, 
   vertical_flip = False)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_image_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary')
print(train_generator.class_indices)

validation_generator = test_datagen.flow_from_directory(
    validation_image_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary')
print(validation_generator.class_indices)

In [ ]:
loss_history=[model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_COUNT,
    epochs=NB_EPOCHS,
    validation_data=validation_generator,
    validation_steps=VAL_IMGS // BATCH_SIZE,
    callbacks=callbacks_list)]



In [ ]:
model.save('vgg_model_best.h5')
model.save_weights('vgg_model_best_w.h5')

In [ ]:
def show_loss(loss_history):
    epich = np.cumsum(np.concatenate(
        [np.linspace(0.5, 1, len(mh.epoch)) for mh in loss_history]))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))
    _ = ax1.plot(epich,
                 np.concatenate([mh.history['loss'] for mh in loss_history]),
                 'b-',
                 epich, np.concatenate(
            [mh.history['val_loss'] for mh in loss_history]), 'r-')
    ax1.legend(['Training', 'Validation'])
    ax1.set_title('Loss')

    _ = ax2.plot(epich, np.concatenate(
        [mh.history['accuracy'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_accuracy'] for mh in loss_history]),
                     'r-')
    ax2.legend(['Training', 'Validation'])
    ax2.set_title('Accuracy')

show_loss(loss_history)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from skimage.io import imread
import tensorflow as tf
import glob
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
from PIL import Image, ImageDraw, ImageFont

IMAGE_WIDTH, IMAGE_HEIGHT = 256, 256

model = tf.keras.models.load_model('vgg_model_best.h5')

file_path = "data/test/"

all_files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

dogs_counter = 0 
cats_counter  = 0
for file in all_files:
    img = image.load_img(file_path+file, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=20)
    classes = classes[0][0]
    
    fontPath = "/Users/Admin/Downloads/open-sans/OpenSans-Semibold.ttf"
    font  =  ImageFont.truetype(fontPath,50)
    size = 400, 300
    
    if classes == 0:
        res_image = Image.open('data/test/'+file)
        draw = ImageDraw.Draw(res_image)
        draw.text((100, 100),"Kitty", font=font, fill=(255,0,0,128))
        res_image.thumbnail(size)
        display(res_image)
        cats_counter += 1
    else:
        res_image = Image.open('data/test/'+file)
        draw = ImageDraw.Draw(res_image)
        draw.text((100, 100),"Doggo", font=font, fill=(255,0,0,128))
        res_image.thumbnail(size)
        display(res_image)
        dogs_counter += 1
print("Total number of Dogs: ",dogs_counter)
print("Total number of Cats: ",cats_counter)